## [Kaggle: Consumer Complaint Database](https://www.kaggle.com/selener/consumer-complaint-database)

- *Download the rows.csv file & save to ../datasets directory before proceeding further*

This Notebook is used to parse the original dataset & prepare a subset of this dataset by compacting some of the product types & retaining 5 product types.

In [53]:
import pandas as pd

In [54]:
original_dataset = pd.read_csv('rows.csv')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (4,5,6,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
original_dataset.head(10)

,Product,Consumer complaint narrative
29904,"Credit reporting, repair, or other",The Summer of XX/XX/2018 I was denied a mortga...
30629,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30735,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30795,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30807,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30857,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30977,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
31013,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
31078,"Credit reporting, repair, or other",There is an account reporting on my credit rep...
31458,"Credit reporting, repair, or other",The reason for my writing is to inform you tha...


In [20]:
original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282355 entries, 0 to 1282354
Data columns (total 2 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Product                       1282355 non-null  object
 1   Consumer complaint narrative  383564 non-null   object
dtypes: object(2)
memory usage: 19.6+ MB


In [21]:
print('Number of Unique Products:', original_dataset.Product.nunique())

Number of Unique Products: 18


In [22]:
original_dataset.Product.unique()

array(['Checking or savings account', 'Debt collection',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Mortgage', 'Student loan', 'Vehicle loan or lease',
       'Credit card or prepaid card',
       'Payday loan, title loan, or personal loan',
       'Money transfer, virtual currency, or money service',
       'Credit reporting', 'Credit card', 'Bank account or service',
       'Consumer Loan', 'Prepaid card', 'Other financial service',
       'Payday loan', 'Money transfers', 'Virtual currency'], dtype=object)

In [23]:
print('Rows with Null Consumer Complaints: ',
      original_dataset[original_dataset['Consumer complaint narrative'].isna()].shape[0])

Rows with Null Consumer Complaints:  898791


In [24]:
print('Original number of rows:', original_dataset.shape[0])
original_dataset.dropna(axis=0, how='any', inplace=True)
print('Remaining number of rows:', original_dataset.shape[0])

Original number of rows: 1282355
Remaining number of rows: 383564


In [30]:
original_dataset.groupby(['Product'])\
    [['Consumer complaint narrative']]\
    .count()\
    .rename({'Consumer complaint narrative': 'Count'}, axis=1)\
    .sort_values('Count', ascending=False)

,Count
Product,
"Credit reporting, repair, or other",123966
Debt collection,86710
Mortgage,52987
Credit card or prepaid card,41667
Student loan,21810
Bank account or service,14885
Checking or savings account,12881
Consumer Loan,9474
"Money transfer, virtual currency, or money service",6979


In [44]:
# Replace Repetitive Product Types. This is to ensure that data is compacted and doesnt contain overlapping classes.

original_dataset.replace(
    {'Product': 
     {'Credit reporting, credit repair services, or other personal consumer reports': 
          'Credit reporting, repair, or other', 
      'Credit reporting': 'Credit reporting, repair, or other',
      'Credit card': 'Credit card or prepaid card',
      'Prepaid card': 'Credit card or prepaid card',
      'Payday loan': 'Payday loan, title loan, or personal loan',
      'Money transfers': 'Money transfer, virtual currency, or money service',
      'Virtual currency': 'Money transfer, virtual currency, or money service'}},
    inplace=True)

In [33]:
original_dataset.groupby(['Product'])\
    [['Consumer complaint narrative']]\
    .count()\
    .rename({'Consumer complaint narrative': 'Count'}, axis=1)\
    .nlargest(20, 'Count')

,Count
Product,
"Credit reporting, repair, or other",123966
Debt collection,86710
Mortgage,52987
Credit card or prepaid card,41667
Student loan,21810
Bank account or service,14885
Checking or savings account,12881
Consumer Loan,9474
"Money transfer, virtual currency, or money service",6979


In [35]:
# Select Different Classes which has some good amount of data & doesnt overlap much.
# This is to keep the problem statement simple.
product_types = [
    'Credit reporting, repair, or other',
    'Debt collection',
    'Student loan',
    'Bank account or service',
    'Money transfer, virtual currency, or money service'
]
sub_dataset = original_dataset[original_dataset['Product'].isin(product_types)].copy()
sub_dataset.shape

(254350, 2)

In [36]:
sub_dataset.groupby(['Product'])[['Consumer complaint narrative']]\
    .count()\
    .rename({'Consumer complaint narrative': 'Count'}, axis=1)\
    .sort_values('Count', ascending=False)

,Count
Product,
"Credit reporting, repair, or other",123966
Debt collection,86710
Student loan,21810
Bank account or service,14885
"Money transfer, virtual currency, or money service",6979


In [62]:
sub_dataset.rename(columns={'Consumer complaint narrative': 'Complaint_text'}, 
                   inplace=True)

In [63]:
sub_dataset.head()

,Product,Complaint_text
29904,"Credit reporting, repair, or other",The Summer of XX/XX/2018 I was denied a mortga...
30629,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30735,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30795,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...
30807,"Credit reporting, repair, or other",There are many mistakes appear in my report wi...


In [67]:
sub_dataset.to_csv('../datasets/consumer_complaints_dataset.csv',
                   columns=['Product', 'Complaint_text'],
                   index=False)